In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import cv2
from pandas.plotting import table
import matplotlib.pyplot as plt
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
tf.enable_eager_execution()
from myclasses import DataGenerator

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [2]:
cte_height = 1080;
cte_width = 1920;

In [3]:
Informacoes = pd.read_csv("Dataset_QP_Certo").drop(columns=["Unnamed: 0"])

In [4]:
Informacoes["Height_Final"] = Informacoes["Height"]+63
Informacoes["Width_Final"] = Informacoes["Width"]+63
Informacoes.drop(Informacoes.loc[(Informacoes["Height_Final"]>cte_height)|(Informacoes["Width_Final"]>cte_width)].index,inplace=True)
Informacoes.drop(columns=['Height_Final',"Width_Final"],inplace=True)
Informacoes.loc[Informacoes['Origem'].str.contains("OldTown",na=False),"Origem"] = "OldTown"
Informacoes.loc[Informacoes['Origem'].str.contains("Crowd",na=False),"Origem"] = "Crowd"
Informacoes.loc[Informacoes['Origem'].str.contains("SunFlower",na=False),"Origem"] = "SunFlower"
Informacoes.loc[Informacoes['Origem'].str.contains("ParkJoy",na=False),"Origem"] = "ParkJoy"
Informacoes.loc[Informacoes['Origem'].str.contains("Ducks",na=False),"Origem"] = "Ducks"
Informacoes.loc[Informacoes['Origem'].str.contains("RushHour",na=False),"Origem"] = "RushHour"

In [5]:
parkjoy = np.fromfile('./Videos/ParkJoy_1920x1080_50_8bit_420.yuv',dtype='uint8')
rushhour= np.fromfile('./Videos/RushHour_1920x1080_25_8bit_420.yuv',dtype='uint8')
oldtown = np.fromfile('./Videos/OldTownCross_1920x1080_50_8bit_420.yuv',dtype='uint8')
ducks = np.fromfile('./Videos/DucksTakeOff_1920x1080_50_8bit_420.yuv',dtype='uint8')
crowdrun = np.fromfile('./Videos/Crowdrun_1920x1080_50_8bit_420.yuv',dtype='uint8')
sunflower = np.fromfile('./Videos/Sunflower_1920x1080_25_8bit_420.yuv',dtype='uint8')

In [6]:
Map = {'OldTown':oldtown,'Crowd':crowdrun,"SunFlower":sunflower,"ParkJoy":parkjoy,"Ducks":ducks,"RushHour":rushhour}

In [7]:
Informacoes

,Origem,Frame,Height,Width,QP,Split
0,Crowd,323,0,0,32,1
1,Crowd,323,0,64,32,1
2,Crowd,323,0,128,32,0
3,Crowd,323,0,192,32,0
4,Crowd,323,0,256,32,0
...,...,...,...,...,...,...
6119965,SunFlower,106,960,1600,37,0
6119966,SunFlower,106,960,1664,37,0
6119967,SunFlower,106,960,1728,37,0
6119968,SunFlower,106,960,1792,37,0


In [8]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
      tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy','mse'])
  return model

model = get_compiled_model()

In [9]:
training_generator = DataGenerator(Informacoes,Map,batch_size=100)

In [10]:
model.fit_generator(generator=training_generator,
                    use_multiprocessing=True,
                    workers=3)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
57600/57600 [==============================] - 3071s 53ms/step - loss: 0.6889 - acc: 0.5460 - mean_squared_error: 0.2481
